In [1]:
from pathlib import Path
import json
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [2]:
def docs_persistence(path: Path) -> str:
    """

    :param path: 爬取文本路径
    :return: 知识库磁盘保存路径
    """
    data_list = []
    for file in path.rglob('*.json'):
        with open(file, 'r', encoding='utf-8-sig') as f:
            data = json.load(f)
            # 格式特殊字符处理
            data['title'] = data['title'].replace('/u200b', '')
            # 保存所有数据
            data_list.append(data)

    # 遍历数据列表，检查文章标题是否存在重复，进行筛选
    all_title = {}
    duplicate_samples_title = []
    all_title['title'] = []
    single_context = []

    for context in data_list:
        all = ''
        for key, value in context.items():
            if type(value) == list:
                for value1 in value:
                    all += value1
            else:
                all += value

            # 根据标题筛选
            if key == 'title' and value not in all_title.values():
                all_title[key].append(value)
            else:
                if key == 'title':
                    duplicate_samples_title.append(value)
        # 合并单个文本信息
        single_context.append(all)
    if duplicate_samples_title:
        print('存在重复标题：', duplicate_samples_title)
    else:
        print('标题均不重复')

    # 所有数据文本信息合并
    all_text = ''

    for text in single_context:
        # 创建 Document 对象

        all_text += text

    # 数据进行递归切割、分块
    text_spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

    split_docs = text_spliter.split_text(all_text)

    # 创建 Document 对象
    # doc = Document(page_content=all_text)
    # split_docs = [Document(page_content=doc) for doc in split_docs]
    # 加载检查点的embedding模型
    persist_directory = r'vectordb/chroma'
    embeddings_model_cache_path = r'autodl-tmp/embedding_model/Ceceliachenen/paraphrase-multilingual-MiniLM-L12-v2'
    embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_cache_path)
    # langchain框架构建本地向量库, 使用chroma知识库

    # 初始化向量数据库
    vectordb = Chroma.from_texts(
        texts=split_docs,
        embedding=embeddings,
        persist_directory=persist_directory,
    )

    vectordb.persist()
    return persist_directory

In [3]:
path = Path.cwd() / 'allinfodata'
docs_persistence(path)


标题均不重复


/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


'vectordb/chroma'